# Capstone Project for IBM Data Science Certificate

##  By Carlos Barros 

### Introduction
The present laboratory is part of the asgination of week 3 of the final project of the professional Big Data certification carried out by IBM through Coursera. In this work explore, segment, and cluster the neighborhoods in the city of Toronto.

The data that will be used is available in the following link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download, Data Setup and Explore Dataset</a>

2. <a href="#item2">Create initial table with 103 postal codes ('Postcode', 'Borough','Neighborhood') </a>

3. <a href="#item3">Concatinate table from part 1 with Geospacial Coordinates ('Latititude', 'Longitude')</a>

4. <a href="#item4">Generate maps to visual neighborhoods and how they cluster using geopy & folium</a>

</font>
</div>




## Download, Data Setup and Explore Dataset

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

!pip install folium
# map rendering library
import folium 

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

print('Libraries imported.')

Libraries imported.


#### Now let's analyze the wikipedia page

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

#### Find the first table on the Wikipedia page and iterate through tags for required information

In [4]:
table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

## Create initial table with 103 Postcodes ('Postcode', 'Borough', 'Neighborhood')

#### Build the dataframe from the list of values

In [5]:
df_can_postcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_can_postcode.columns = ['Postcode', 'Borough', 'Neighborhood']
df_can_postcode.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Cleanse the data and transform as per the given requirements

In [6]:
#data transform - if 'Neighborhood' = 'Not Assigned', then use 'Borough'
df_can_postcode.drop(df_can_postcode[df_can_postcode['Borough'] == 'Not assigned'].index, inplace=True)
df_can_postcode.loc[df_can_postcode.Neighborhood == 'Not assigned', "Neighborhood"] = df_can_postcode.Borough
df_can_postcode.head(5)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Group the data by Postcode & Borough

In [7]:
#Group the dataframe by Postcode & Borough and 'Join' values in 'Neighborhood'
df_grp_can = df_can_postcode.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grp_can.columns = ['Postcode', 'Borough', 'Neighborhood']
df_grp_can.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
